In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter, HourLocator
from matplotlib.gridspec import GridSpec
from datetime import datetime, timedelta
from glob import glob
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings("ignore")
from ipywidgets import interact, fixed

%matplotlib inline
plt.rcParams['font.sans-serif'] = ['SimHei']  # 解决中文显示问题
plt.rcParams['axes.unicode_minus'] = False    # 解决负号显示问题

In [2]:
df = pd.read_excel("../datas/打螺丝数据.xlsx")

In [3]:
df

,Angle,Torque,文件名,螺丝号,日期,时间,工序,位置,版本
0,0.00,0.00000,F9WHHK000QQ0000X7U,pos41ScrewNum1,2025-06-28,13074770,LA6SPKCHGTOHSG,Loc41,Full_Config_V4
1,0.00,0.00000,F9WHHK000QQ0000X7U,pos41ScrewNum1,2025-06-28,13074770,LA6SPKCHGTOHSG,Loc41,Full_Config_V4
2,0.00,0.00000,F9WHHK000QQ0000X7U,pos41ScrewNum1,2025-06-28,13074770,LA6SPKCHGTOHSG,Loc41,Full_Config_V4
3,0.00,0.00000,F9WHHK000QQ0000X7U,pos41ScrewNum1,2025-06-28,13074770,LA6SPKCHGTOHSG,Loc41,Full_Config_V4
4,0.00,0.00000,F9WHHK000QQ0000X7U,pos41ScrewNum1,2025-06-28,13074770,LA6SPKCHGTOHSG,Loc41,Full_Config_V4
...,...,...,...,...,...,...,...,...,...
29295,946.08,0.15665,F9WHHK000QQ0000X7U,pos4ScrewNum1,2025-06-28,13072550,LA6SPKCHGTOHSG,Loc4,Half_Config_V3
29296,947.70,0.16210,F9WHHK000QQ0000X7U,pos4ScrewNum1,2025-06-28,13072550,LA6SPKCHGTOHSG,Loc4,Half_Config_V3
29297,948.78,0.16720,F9WHHK000QQ0000X7U,pos4ScrewNum1,2025-06-28,13072550,LA6SPKCHGTOHSG,Loc4,Half_Config_V3
29298,950.40,0.16740,F9WHHK000QQ0000X7U,pos4ScrewNum1,2025-06-28,13072550,LA6SPKCHGTOHSG,Loc4,Half_Config_V3


In [4]:
df.columns

Index(['Angle', 'Torque', '文件名', '螺丝号', '日期', '时间', '工序', '位置', '版本'], dtype='object')

In [5]:
df['文件名'].unique()

array(['F9WHHK000QQ0000X7U', 'F9WHHH0003V0000X7U', 'F9WHHH0004X0000X7U',
       'F9WHHH0005S0000X7U', 'F9WHHH0006X0000X7U', 'F9WHHH000450000X7U',
       'F9WHHH000480000X7U', 'F9WHHH000490000X7U', 'F9WHHH000500000X7U',
       'F9WHHH000730000X7U'], dtype=object)

In [18]:
single_file = df[df['文件名'] == 'F9WHHK000QQ0000X7U']
single_screwnum = single_file[single_file['螺丝号'] == single_file['螺丝号'].unique()[3]]
print(single_screwnum[['Angle', 'Torque']].corr())
single_screwnum

           Angle    Torque
Angle   1.000000  0.788945
Torque  0.788945  1.000000


,Angle,Torque,文件名,螺丝号,日期,时间,工序,位置,版本,date,time,完整时间
28615,0.00,0.00000,F9WHHK000QQ0000X7U,pos4ScrewNum1,2025-06-28,13072550,LA6SPKCHGTOHSG,Loc4,Half_Config_V3,2025-06-28,1 days 12:18:45,2025-06-29 12:18:45
28616,0.00,0.00000,F9WHHK000QQ0000X7U,pos4ScrewNum1,2025-06-28,13072550,LA6SPKCHGTOHSG,Loc4,Half_Config_V3,2025-06-28,1 days 12:18:45,2025-06-29 12:18:45
28617,0.00,0.00000,F9WHHK000QQ0000X7U,pos4ScrewNum1,2025-06-28,13072550,LA6SPKCHGTOHSG,Loc4,Half_Config_V3,2025-06-28,1 days 12:18:45,2025-06-29 12:18:45
28618,0.00,0.00000,F9WHHK000QQ0000X7U,pos4ScrewNum1,2025-06-28,13072550,LA6SPKCHGTOHSG,Loc4,Half_Config_V3,2025-06-28,1 days 12:18:45,2025-06-29 12:18:45
28619,0.00,0.00000,F9WHHK000QQ0000X7U,pos4ScrewNum1,2025-06-28,13072550,LA6SPKCHGTOHSG,Loc4,Half_Config_V3,2025-06-28,1 days 12:18:45,2025-06-29 12:18:45
...,...,...,...,...,...,...,...,...,...,...,...,...
29295,946.08,0.15665,F9WHHK000QQ0000X7U,pos4ScrewNum1,2025-06-28,13072550,LA6SPKCHGTOHSG,Loc4,Half_Config_V3,2025-06-28,1 days 12:18:45,2025-06-29 12:18:45
29296,947.70,0.16210,F9WHHK000QQ0000X7U,pos4ScrewNum1,2025-06-28,13072550,LA6SPKCHGTOHSG,Loc4,Half_Config_V3,2025-06-28,1 days 12:18:45,2025-06-29 12:18:45
29297,948.78,0.16720,F9WHHK000QQ0000X7U,pos4ScrewNum1,2025-06-28,13072550,LA6SPKCHGTOHSG,Loc4,Half_Config_V3,2025-06-28,1 days 12:18:45,2025-06-29 12:18:45
29298,950.40,0.16740,F9WHHK000QQ0000X7U,pos4ScrewNum1,2025-06-28,13072550,LA6SPKCHGTOHSG,Loc4,Half_Config_V3,2025-06-28,1 days 12:18:45,2025-06-29 12:18:45


,Angle,Torque
Angle,1.000000,0.057732
Torque,0.057732,1.000000


In [9]:
# 转换日期和时间列
# 注意：Excel中的日期是数值格式，需转datetime
df['date'] = pd.to_datetime(df['日期'])
df['time'] = pd.to_timedelta(df['时间'].astype(str).str[:6] + ' seconds')

# 创建完整时间戳 = 日期 + 时间
df['完整时间'] = df['date'] + df['time']

In [ ]:
# 可视化函数
def visualize_group(group, filename):
    plt.figure(figsize=(12, 9))
    plt.suptitle(f"文件分析: {filename}", fontsize=14)
    
    # 1. 扭矩分布直方图
    plt.subplot(2, 2, 1)
    sns.histplot(group['Torque'], kde=True, color='#4285F4', 
                 element='step', stat='density', alpha=0.7)
    plt.title('扭矩分布', fontsize=10)
    plt.xlabel('扭矩值 (Nm)', fontsize=9)
    plt.ylabel('密度', fontsize=9)
    plt.grid(axis='y', alpha=0.3)
    
    # 2. 角度-扭矩关系散点图
    plt.subplot(2, 2, 2)
    plt.scatter(
        group['Angle'], group['Torque'], 
        alpha=0.15, color='#4285F4',
        s=10
    )
    sns.regplot(
        x='Angle', y='Torque', data=group, 
        scatter=False, color='#EA4335',
        lowess=True
    )
    plt.title('角度-扭矩关系', fontsize=10)
    plt.xlabel('角度 (°)', fontsize=9)
    plt.ylabel('扭矩 (Nm)', fontsize=9)
    plt.grid(alpha=0.3)
    
    # 3. 分位置扭矩分布
    plt.subplot(2, 2, 3)
    position_order = group.groupby('位置')['Torque'].mean().sort_values().index
    sns.violinplot(
        x='位置', y='Torque', data=group,
        inner='quartile',
        order=position_order,
        palette='Blues'
    )
    plt.title('不同位置扭矩分布', fontsize=10)
    plt.xlabel('位置编号', fontsize=9)
    plt.ylabel('扭矩 (Nm)', fontsize=9)
    plt.grid(axis='y', alpha=0.3)
    
    # 4. 重构的时间序列扭矩变化图
    plt.subplot(2, 2, 4)
    
    # 按时间排序并添加连续序列号
    sorted_group = group.sort_values('完整时间').reset_index(drop=True)
    sorted_group['序列号'] = np.arange(len(sorted_group))
    
    # 解决错误：使用滚动窗口代替分组
    window_size = max(1, len(sorted_group) // 20)  # 自适应窗口大小
    
    # 计算移动平均值和标准差
    sorted_group['扭矩移动平均'] = sorted_group['Torque'].rolling(
        window=window_size, min_periods=1).mean()
    
    sorted_group['扭矩移动标准差'] = sorted_group['Torque'].rolling(
        window=window_size, min_periods=1).std()
    
    sorted_group['角度移动平均'] = sorted_group['Angle'].rolling(
        window=window_size, min_periods=1).mean()
    
    # 获取当前轴
    ax1 = plt.gca()
    
    # 扭矩趋势线（主坐标轴）
    ax1.plot(
        sorted_group['序列号'], sorted_group['扭矩移动平均'],
        '-', color='#4285F4', 
        linewidth=2,
        alpha=0.9, label='扭矩移动平均'
    )
    
    # 扭矩波动范围
    ax1.fill_between(
        sorted_group['序列号'],
        sorted_group['扭矩移动平均'] - 0.5 * sorted_group['扭矩移动标准差'],
        sorted_group['扭矩移动平均'] + 0.5 * sorted_group['扭矩移动标准差'],
        alpha=0.2, color='#4285F4', label='扭矩波动范围'
    )
    
    ax1.set_xlabel('操作序列', fontsize=9)
    ax1.set_ylabel('扭矩 (Nm)', color='#4285F4', fontsize=9)
    ax1.tick_params(axis='y', labelcolor='#4285F4')
    
    # 角度趋势线（次坐标轴）
    ax2 = ax1.twinx()
    ax2.plot(
        sorted_group['序列号'], sorted_group['角度移动平均'],
        '-', color='#EA4335', 
        linewidth=1.5,
        alpha=0.7, label='角度移动平均'
    )
    
    ax2.set_ylabel('角度 (°)', color='#EA4335', fontsize=9)
    ax2.tick_params(axis='y', labelcolor='#EA4335')
    
    # 合并图例
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left', fontsize=8)
    
    plt.title('时间序列扭矩变化', fontsize=10)
    ax1.grid(alpha=0.2)
    
    # 整体优化布局
    plt.tight_layout(pad=2.0)  # 增加内边距
    plt.savefig(f"../datas/imgs/screw/{filename}_analysis.png", dpi=150, bbox_inches='tight')
    plt.close()

In [34]:
# 可视化函数
def visualize_group(group, filename):
    plt.figure(figsize=(12, 9))
    plt.suptitle(f"文件分析: {filename}", fontsize=14)
    
    screw_ids = group['位置'].unique()
    # 1. 扭矩分布直方图
    plt.subplot(2, 2, 1)
    for screw_id in screw_ids:
        screw_data = group[group['位置'] == screw_id]
        sns.histplot(screw_data['Torque'], kde=True, element='step', stat='density', alpha=0.7, label=screw_id)     # , color='#4285F4'
    plt.title('扭矩分布', fontsize=10)
    plt.xlabel('扭矩值 (Nm)', fontsize=9)
    plt.ylabel('密度', fontsize=9)
    plt.legend()
    plt.grid(axis='y', alpha=0.3)
    
    # 2. 角度-扭矩关系散点图
    plt.subplot(2, 2, 2)
    for screw_id in screw_ids:
        screw_data = group[group['位置'] == screw_id]
        plt.scatter(screw_data['Angle'], screw_data['Torque'], alpha=0.7, s=15, label=screw_id)
        # 添加趋势线
        sns.regplot(x='Angle', y='Torque', data=screw_data, scatter=False, lowess=True)
        
    plt.title(f'角度-扭矩关系')
    plt.xlabel('角度 (°)', fontsize=9)
    plt.ylabel('扭矩 (Nm)', fontsize=9)
    plt.grid(alpha=0.3)
    plt.legend()
    
    # 3. 分位置扭矩分布
    plt.subplot(2, 2, 3)
    position_order = group.groupby('位置')['Torque'].mean().sort_values().index
    sns.violinplot(x='位置', y='Torque', data=group, inner='quartile', order=position_order, palette='Blues')
    plt.title('不同位置扭矩分布', fontsize=10)
    plt.xlabel('位置编号', fontsize=9)
    plt.ylabel('扭矩 (Nm)', fontsize=9)
    plt.grid(axis='y', alpha=0.3)
    
    # 4. 重构的时间序列扭矩变化图
    plt.subplot(2, 2, 4)
    
    # 按时间排序并添加连续序列号
    sorted_group = group.sort_values('完整时间').reset_index(drop=True)
    sorted_group['序列号'] = np.arange(len(sorted_group))
    
    # 解决错误：使用滚动窗口代替分组
    window_size = max(1, len(sorted_group) // 20)  # 自适应窗口大小
    
    # 计算移动平均值和标准差
    sorted_group['扭矩移动平均'] = sorted_group['Torque'].rolling(window=window_size, min_periods=1).mean()
    sorted_group['扭矩移动标准差'] = sorted_group['Torque'].rolling(window=window_size, min_periods=1).std()
    sorted_group['角度移动平均'] = sorted_group['Angle'].rolling(window=window_size, min_periods=1).mean()
    # 获取当前轴
    ax1 = plt.gca()
    # 扭矩趋势线（主坐标轴）
    ax1.plot(sorted_group['序列号'], sorted_group['扭矩移动平均'], '-', color='#4285F4', linewidth=2, alpha=0.9, label='扭矩移动平均')
    
    # 扭矩波动范围
    ax1.fill_between(
        sorted_group['序列号'],
        sorted_group['扭矩移动平均'] - 0.5 * sorted_group['扭矩移动标准差'],
        sorted_group['扭矩移动平均'] + 0.5 * sorted_group['扭矩移动标准差'],
        alpha=0.2, color='#4285F4', label='扭矩波动范围'
    )
    
    ax1.set_xlabel('操作序列', fontsize=9)
    ax1.set_ylabel('扭矩 (Nm)', color='#4285F4', fontsize=9)
    ax1.tick_params(axis='y', labelcolor='#4285F4')
    
    # 角度趋势线（次坐标轴）
    ax2 = ax1.twinx()
    ax2.plot(sorted_group['序列号'], sorted_group['角度移动平均'], '-', color='#EA4335', linewidth=1.5, alpha=0.7, label='角度移动平均')
    
    ax2.set_ylabel('角度 (°)', color='#EA4335', fontsize=9)
    ax2.tick_params(axis='y', labelcolor='#EA4335')
    
    # 合并图例
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left', fontsize=8)
    
    plt.title('时间序列扭矩变化', fontsize=10)
    ax1.grid(alpha=0.2)
    
    # 整体优化布局
    plt.tight_layout(pad=2.0)  # 增加内边距
    plt.savefig(f"../datas/imgs/screw/{filename}_analysis.png", dpi=150, bbox_inches='tight')
    plt.close()

In [35]:
# 按文件名分组分析
def analyze_by_filename(df):
    results = {}
    
    for filename, group in df.groupby('文件名'):
        # 基本统计
        stats = {
            '文件样本数': len(group),
            '唯一螺丝号': group['螺丝号'].nunique(),
            '工序类型': list(group['工序'].unique()),
            '使用版本': list(group['版本'].unique()),
            
            '扭矩均值': group['Torque'].mean(),
            '扭矩标准差': group['Torque'].std(),
            '扭矩最大值': group['Torque'].max(),
            '扭矩最小值': group['Torque'].min(),
            '扭矩25%分位数': group['Torque'].quantile(0.25),
            '扭矩75%分位数': group['Torque'].quantile(0.75),
            
            '角度均值': group['Angle'].mean(),
            '角度标准差': group['Angle'].std(),
        }
        
        # 螺丝位置分布
        position_stats = group['位置'].value_counts().to_dict()
        
        # 时间范围
        time_range = {
            '开始时间': group['完整时间'].min(),
            '结束时间': group['完整时间'].max(),
            '持续时间(秒)': (group['完整时间'].max() - group['完整时间'].min()).total_seconds()
        }
        
        results[filename] = {
            '基本统计': stats,
            '位置分布': position_stats,
            '时间统计': time_range
        }
        
        # 生成可视化 (每文件单独保存)
        visualize_group(group, filename)
    
    return results

In [36]:
# 执行分析
analysis_results = analyze_by_filename(df)

In [25]:
# 输出统计摘要
summary_df = pd.DataFrame({
    '文件名': [k for k in analysis_results],
    '样本数': [v['基本统计']['文件样本数'] for k, v in analysis_results.items()],
    '螺丝数': [v['基本统计']['唯一螺丝号'] for k, v in analysis_results.items()],
    '平均扭矩': [v['基本统计']['扭矩均值'] for k, v in analysis_results.items()],
    '最大扭矩': [v['基本统计']['扭矩最大值'] for k, v in analysis_results.items()],
    '工作时长(秒)': [v['时间统计']['持续时间(秒)'] for k, v in analysis_results.items()]
})

print("文件汇总统计:")
print(summary_df)

文件汇总统计:
                  文件名   样本数  螺丝数      平均扭矩     最大扭矩  工作时长(秒)
0  F9WHHH0003V0000X7U  3078    4  0.061758  0.34355     62.0
1  F9WHHH000450000X7U  3011    4  0.065026  0.34375     63.0
2  F9WHHH000480000X7U  2948    4  0.062211  0.34420     62.0
3  F9WHHH000490000X7U  2726    4  0.052891  0.34390     22.0
4  F9WHHH0004X0000X7U  2793    4  0.050552  0.34375     21.0
5  F9WHHH000500000X7U  3068    4  0.066826  0.34370     62.0
6  F9WHHH0005S0000X7U  2782    4  0.051420  0.34330     22.0
7  F9WHHH0006X0000X7U  3148    4  0.063427  0.34350     21.0
8  F9WHHH000730000X7U  2924    4  0.048271  0.34330     22.0
9  F9WHHK000QQ0000X7U  2822    4  0.051777  0.34335     22.0


In [26]:
# 保存详细结果
with pd.ExcelWriter('螺丝分析报告.xlsx') as writer:
    summary_df.to_excel(writer, sheet_name='文件汇总', index=False)
    
    detail_data = []
    for filename, data in analysis_results.items():
        detail_data.append({
            '文件名': filename,
            **data['基本统计'],
            **{'位置_' + str(k): v for k, v in data['位置分布'].items()}
        })
    
    pd.DataFrame(detail_data).to_excel(writer, sheet_name='详细统计', index=False)
    
    # 添加原始数据
    df.to_excel(writer, sheet_name='原始数据', index=False)

print("分析完成！生成报告: 螺丝分析报告.xlsx")
print("各文件分析图已保存为 [文件名]_analysis.png")

分析完成！生成报告: 螺丝分析报告.xlsx
各文件分析图已保存为 [文件名]_analysis.png
